In [1]:
import pandas as pd   
import math
import time
import librosa
import whisper
from pydub import AudioSegment
import numpy as np 
import torch
import torch.nn as nn

In [1718]:

model = whisper.load_model("small")


audio_path = "/Users/withmocha/Desktop/DATA/Capston Design(2024)/data(sentence)/0518(final)/detection test data/voice fishing/기관 사칭/기관사칭5.mp3"


result = model.transcribe(audio_path)


print(result["text"])

/opt/anaconda3/envs/conda_cpu/lib/python3.12/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 준 준비할 때 OOC의 이번 사건에 그 본인 통장이 어떻게 좀 개설이 됐을까요? 본인 통장이 본인이 개설을 안 하셨다고 진술을 해주신 것 같아요. 본인이 개설 안 했으면 타인이 개설을 하려고 하면 등본이라든가 가족 관계 증명서를 제출을 해야 통장을 만들어줘요.


In [1719]:
import time
start = time.time()

result["text"]

' 준 준비할 때 OOC의 이번 사건에 그 본인 통장이 어떻게 좀 개설이 됐을까요? 본인 통장이 본인이 개설을 안 하셨다고 진술을 해주신 것 같아요. 본인이 개설 안 했으면 타인이 개설을 하려고 하면 등본이라든가 가족 관계 증명서를 제출을 해야 통장을 만들어줘요.'

In [1720]:
A=len(result["text"])

In [1721]:
A

146

In [1722]:

audio = AudioSegment.from_mp3(audio_path)

print(f"Channels: {audio.channels}")
print(f"Sample Width: {audio.sample_width}")
print(f"Frame Rate (Sample Rate): {audio.frame_rate}")
print(f"Frame Width: {audio.frame_width}")
print(f"Length (ms): {len(audio)}")

Channels: 1
Sample Width: 2
Frame Rate (Sample Rate): 24000
Frame Width: 2
Length (ms): 24984


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1723]:
seconds=(len(audio)/1000.0)

In [1724]:
seconds# 초

24.984

In [1725]:
step_cost=seconds/A
step_cost

0.1711232876712329

In [1726]:
time=0
length=0
for i in range(A):
    time=time+step_cost
    length=length+1
    if(time>30):# 녹음 시간
        break

In [1727]:
print(time)
print(length)

24.983999999999952
146


In [1728]:
target_data=pd.DataFrame(index=range(1),columns=['sentence'])

In [1729]:
target_data.iloc[0,0]=result['text']

In [1730]:
target_data

,sentence
0,준 준비할 때 OOC의 이번 사건에 그 본인 통장이 어떻게 좀 개설이 됐을까요? ...


In [1731]:
target_data.to_csv("data.csv")

In [1732]:

count_s = 0
B = []
original_str = target_data.iloc[0,0]
A = len(original_str)

for j in range(A):
    char = original_str[j]
    if char not in [',', '\'', '[', ']']:
        B.append(char)
    if len(B)>length:
        break
       

new_str = ''.join(B)
target_data.iloc[0,0] = new_str

In [1733]:
target_data.iloc[0,0]

' 준 준비할 때 OOC의 이번 사건에 그 본인 통장이 어떻게 좀 개설이 됐을까요? 본인 통장이 본인이 개설을 안 하셨다고 진술을 해주신 것 같아요. 본인이 개설 안 했으면 타인이 개설을 하려고 하면 등본이라든가 가족 관계 증명서를 제출을 해야 통장을 만들어줘요.'

In [1734]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('jhgan/ko-sbert-sts')


In [1735]:
A=model.encode(target_data.iloc[0,0])

In [1736]:
list=[]

for i in range(A.shape[0]):
    list.append(i)

In [1737]:
x_train=pd.DataFrame(index=range(1),columns=list)

In [1738]:
x_train

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1739]:
vector=[]
vector=model.encode(target_data.iloc[0,0])  
for j in range(x_train.shape[1]):
        x_train.iloc[0,j]=vector[j]  
    

In [1740]:
x_train

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.459314,1.452404,-0.443741,0.720802,-0.612723,-0.600675,0.068627,0.735096,-0.462058,-0.840374,...,-0.002946,1.233805,0.48415,1.730471,-0.544479,-0.078419,-0.19546,0.105926,0.086356,-0.078995


In [1741]:
X_train = torch.tensor(x_train.iloc[0], dtype=torch.float).unsqueeze(1)

In [1742]:
X_train.shape

torch.Size([768, 1])

In [1743]:
X_train = X_train.reshape(1, 1, X_train.shape[0])

In [1744]:
X_train.shape

torch.Size([1, 1, 768])

In [1745]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=X_train.shape[2], out_channels=128, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=1)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=1)
        self.pool = nn.MaxPool1d(kernel_size=1)
        self.dropout = nn.Dropout(0.5)  
        self.lstm = nn.LSTM(input_size=32, hidden_size=128, num_layers=2, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(128*2, 32)
        self.fc2 = nn.Linear(32, 3)
        self.activation = nn.ELU()
        self.bn1 = nn.BatchNorm1d(128)  
        self.bn2 = nn.BatchNorm1d(64)
        self.bn3 = nn.BatchNorm1d(32)

    def forward(self, x):
        x = self.bn1(self.activation(self.conv1(x)))
        x = self.bn2(self.activation(self.conv2(x)))
        x = self.bn3(self.activation(self.conv3(x)))
        x = self.pool(x)
        x = self.dropout(x)
        x = x.permute(0, 2, 1)  
        x, _ = self.lstm(x)
        x = self.dropout(x[:, -1, :])  
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [1746]:
model = MyModel()

In [1747]:
state_dict = torch.load("/Users/withmocha/Desktop/DATA/Capston Design(2024)/data(sentence)/0518(final)/model/voice phishing detection model(0521).pt")
model.load_state_dict(state_dict)

<All keys matched successfully>

In [1748]:
model.eval()

MyModel(
  (conv1): Conv1d(768, 128, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
  (conv3): Conv1d(64, 32, kernel_size=(1,), stride=(1,))
  (pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (lstm): LSTM(32, 128, num_layers=2, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=256, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=3, bias=True)
  (activation): ELU(alpha=1.0)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [1749]:

X_train = X_train.transpose(1, 2)
y_score = model(X_train).detach().numpy()

In [1750]:
y_score

array([[-6.664891 , -6.6098433,  8.190697 ]], dtype=float32)

In [1751]:
y_pred = np.argmax(y_score, axis=1)

In [1752]:
y_pred

array([2])

In [1753]:
import time
end = time.time()
print(f"{end - start:.4f} sec")

3.7296 sec
